## Image processing using Python and its environment (FlyScan)

---
### 1) Manage paths with pathlib

Here we will use the [pathlib library](https://docs.python.org/3/library/pathlib.html) to simplify path management and avoid incompatibility between operating systems.  

First, we will import the library and then check the path of the current directory using `Path.cwd()`:

In [1]:
from pathlib import Path
print(Path.cwd())

E:\3-GitHub_BDehapiot\Python_ImageProcessing_2022


This should output the complete path toward the downloaded "Python_ImageProcessing_2022" directory.  
If not, you probably started the notebook from the wrong directory.

Next, we would like to check if the "fly_images" folder exists within the current directory.  
To do that, we need to first create a new path by combining the current path with the folder name:

In [2]:
print(Path('fly_images'))

fly_images


Since pathlib handles paths as objects and not as strings, you cannot directly output the newly created path using the `print` function.  
We will use the pathlib `resolve()` method instead:

In [3]:
print(Path('fly_images').resolve())

E:\3-GitHub_BDehapiot\Python_ImageProcessing_2022\fly_images


Now, to check if the newly created path lead to an existing folder, we will use the pathlib `exists()` method:

In [4]:
print(Path('fly_images').exists())

True


Finally, we will create two lists containing 1) paths and 2) names of the files contain in the "fly_images" folder.   
This can be done by iterating through "fly_images" using the pathlib `iterdir()` method: 

In [43]:
image_paths = []
image_names = []
for path in Path('fly_images').iterdir():
    image_paths.append(path)
    image_names.append(path.name) # name attribute of the path object

**Exercice 1**  
1) Create folders that do not already exist
2) Displace image files using pathlib

---
### 2) Import images 

In [37]:
from skimage import io
im_first = io.imread(image_paths[0])

In [40]:
print(im_first.shape)
im_height = im_first.shape[0]
im_width = im_first.shape[1]

(480, 640)


In [46]:
im_count = len(image_paths)
print(im_count)

200


**Exercice 2**

In [48]:
import numpy as np

In [52]:
# Correction
im_all = np.zeros([im_count, im_height, im_width], dtype='uint8')
for i, path in enumerate(image_paths):   
    im_all[i,...] = io.imread(path)  

In [50]:
import napari
viewer = napari.view_image(im_all)

---
### 3) Process images 